# 如何解析 YAML 输出

:::info 前置条件

本指南假设您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)
- [提示模板](/docs/concepts/prompt_templates)
- [结构化输出](/docs/how_to/structured_output)
- [链接 runnables](/docs/how_to/sequence/)

:::

不同提供商的 LLM 根据其训练数据的不同，通常在特定方面具有不同的优势。这也意味着有些模型在生成 JSON 以外的格式时可能会 "更好" 且更可靠。

这个输出解析器允许用户指定任意模式，并查询 LLM 以获取符合该模式的输出，使用 YAML 来格式化它们的响应。

:::note
请记住，大型语言模型是不完全可靠的抽象！您需要使用具有足够能力生成格式良好的 YAML 的 LLM。
:::


In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

我们使用 [Pydantic](https://docs.pydantic.dev) 和 [`YamlOutputParser`](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.yaml.YamlOutputParser.html#langchain.output_parsers.yaml.YamlOutputParser) 来声明我们的数据模型，并为模型提供更多关于应该生成什么类型的 YAML 的上下文：

In [ ]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# 定义您想要的数据结构
class Joke(BaseModel):
    setup: str = Field(description="用于设置笑话的问题")
    punchline: str = Field(description="解开笑话的答案")


model = ChatOpenAI(temperature=0)

# 一个旨在提示语言模型填充数据结构的查询
joke_query = "讲个笑话。"

# 设置一个解析器并将指令注入到提示模板中
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="回答用户查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why couldn't the bicycle find its way home?", punchline='Because it lost its bearings!')

解析器会自动解析输出的 YAML 并使用数据创建一个 Pydantic 模型。我们可以看到解析器的 `format_instructions`，它们会被添加到提示中：

In [5]:
parser.get_format_instructions()

'The output should be formatted as a YAML instance that conforms to the given JSON schema below.\n\n# Examples\n## Schema\n```\n{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}\n```\n## Well formatted instance\n```\n- name: John Doe\n  avg: 0.3\n- name: Jane Maxfield\n  avg: 1.4\n```\n\n## Schema\n```\n{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}\n```\n## Well formatted instance\n```\nhabit: Using disposable water bottles for daily hydration.\nsustainabl

您可以也应该尝试在提示的其他部分添加自己的格式提示，以增强或替换默认指令。

## 下一步

您现在已经学会了如何提示模型返回 YAML。接下来，查看[获取结构化输出的更广泛指南](/docs/how_to/structured_output)以了解其他相关技术。